# 고급 알고리즘(Advanced): Ensemble 방법론

> **앙상블(Ensemble, Ensemble Learning, Ensemble Method)을 활용한 머신러닝 알고리즘**
> - 여러개의 알고리즘을 학습시킨 후 각각의 예측결과 모두를 사용하여 하나의 알고리즘보다 더 나은 예측성능을 달성
>> - **Bagging:** 데이터를 여러개의 샘플로 나눈 뒤, 각 샘플들을 병렬적으로 학습하여 평균치로 성능 향상
>> - **Boosting:** 데이터를 여러개의 샘플로 나눈 뒤, 학습 에러에 가중치 높여 지속 학습하여 성능 향상
>> - **Stacking:** 여러개의 알고리즘 결과를 새로운 변수로 반영하는 데이터셋을 생성

---

<center><br>"Train & Test 모두 최소 Bias & Variance가 Best"</br></center>
<center><img src='Image/Bias_Variance1.jpeg' width='400'></center>

    
<!-- **1) 적합력에 따른 편향과 분산의 관계:**

> - **Under-fitting: 알고리즘의 복잡도가 낮아 Bias가 증가하고 Variance가 감소**  
>> **Training/Testing 모두 추정/예측 성능이 낮음**
>> - Training: Mid Bias + Low Variance
>> - Testing: High Bias + Mid Variance
> - **Over-fitting: 알고리즘의 복잡도가 높아 Bias가 감소하고 Variance가 증가**  
>> **Training만 추정 성능이 높고 Testing 예측 성능은 낮음**
>> - Training: Low Bias + Mid Variance
>> - Testing: Mid Bias + High Variance -->

<!-- <center><img src='Image/Bias_Variance4.png' width='400'></center> -->

- **편향과 분산 모두를 최소화하는 방법**
<center><img src='Image/Bias_Variance_Reduce.png' width='600'></center>

## Bagging 알고리즘: Variance 줄이기 위한 목적

> - 데이터를 복원샘플링하고 모든 feature를 동등하게 학습하여 평균치를 최종결과로 선정
> - **단점:** 
>> - 학습하지 못한 패턴들은 계속 학습하지 못함
> - **장덤:** 
>> - 여러개의 알고리즘 결과의 평균을 사용하기에 결과가 안정적
>> - 각 샘플들을 병렬로 학습하기 때문에 빠름
>> - 높은 Bias로 인한 Underfitting과 높은 Variance로 인한 Overfitting 문제를 해결하는데 도움

<center><img src='Image/Ensemble_Bagging.jpg' width='400'></center> 

---

**1) 의사결정나무(Decision Tree):**  

<center><img src='Image/Bagging_DT.png' width='700'></center> 

**2) 렌덤포레스트(Random Forest):** 
> - 여러개의 의사결정나무(Decision Tree) 알고리즘으로 추정한 다음, 각 트리의 예측값들 중 가장 영향력이 높은 변수들로 예측
> - 의사결정나무의 CLT(Central Limit Theory)버전

<center><img src='Image/Bagging_RF.jpg' width='700'></center> 

---

```python
# DecisionTree
fit = DecisionTreeRegressor().fit(X_train, Y_train)
pred_tr = fit.predict(X_train)
pred_te = fit.predict(X_test)

# RandomForestRegressor
fit = RandomForestRegressor(n_estimators=100, random_state=123).fit(X_train, Y_train)
pred_tr = fit.predict(X_train)
pred_te = fit.predict(X_test)
```

## Boosting 알고리즘: Bias 줄이기 위한 목적

> - 전체데이터를 처음에 학습하되, 학습하지 못한 feature 특징을 점점 더 중요하게 여겨 이를 지속적으로 학습
> - **단점:** 
>> - 잘 학습된 특징들은 고려하지 않음
>> - 학습하지 못한 데이터들이 생길때마다 학습하기 때문에 직렬처리로 느린 편
> - **장덤:** 
>> - Bagging과 달리 맞추기 어려운 문제를 학습하는데 특화
>> - 학습되지 않은 패턴을 계속 반영하기 Bias를 줄이는데 효과적
>> - Outlier나 Anomaly Detection에 강함

<center><img src='Image/Ensemble_Boosting.jpg' width='400'></center> 

---

**1) Adaptive Boosting(AdaBoost):** 
> - 학습 시 오류가 발생한 데이터에 가중치를 높여 학습을 반복
> - 최종적으로 계산된 가중치들을 합산하여 최종 알고리즘으로 제안

<center><img src='Image/Boosting_AdaBoost.png' width='600'></center> 

**2) Gradient Boosting Machine(GBM):** 
> - AdaBoost과 달리 특정 데이터의 가중치 업데이트 대신 잔차(Residual)를 재학습시키는 방법
> - 재학습시에 더욱 복잡한 Tree 관련 알고리즘을 사용하기에 Overfitting 위험 존재
> - XGBoost, LightGBM을 파생시킨 알고리즘
> ```python
fit = GradientBoostingRegressor(alpha=0.1, learning_rate=0.05, loss='huber', criterion='friedman_mse',
                                           n_estimators=1000, random_state=123).fit(X_train, Y_train)
Y_trpred = fit.predict(X_train)
Y_tepred = fit.predict(X_test)
> ```

<center><img src='Image/Boosting_GBM.png' width='600'></center> 

**3) XGBoost(eXtreme Gradient Boosting):** 
> - Gradient Boosting 알고리즘의 Overfitting 문제를 보완하기 위해 개발
> - Gradient Boosting + Regularization 형태로 다양한 Loss Function 지원
> - 높은 예측력으로 많은 양의 데이터를 다룰 때 사용되는 부스팅 알고리즘  
>```python
fit = XGBRegressor(learning_rate=0.05, n_estimators=100, random_state=123).fit(X_train, Y_train)
Y_trpred = fit.predict(X_train)
Y_tepred = fit.predict(X_test)
>```

<center><img src='Image/Boosting_XGBoost.png' width='600'></center> 

**4) LightGBM:** 
> - Gradient Boosting은 모든 데이터를 탐색/학습하기에 학습 시간이 오래걸리는 문제를 보완하기 위해 개발
> - 모든 데이터 대신 Histogram의 구분점들에서만 탐색/학습하기 때문에 학습시간 단축
> - 현존하는 부스팅 알고리즘 중 가장 빠르고 높은 예측력 제공
> ```python
fit = LGBMRegressor(learning_rate=0.05, n_estimators=100, random_state=123).fit(X_train, Y_train)
Y_trpred = fit.predict(X_train)
Y_tepred = fit.predict(X_test)
>```

<center><img src='Image/Boosting_LightGBM.png' width='600'></center> 


- **비교 정리:**

| Algorithms | Specification | Others |
|------------|-------------------------------------------------------------------------------------------------------------------------------------------|-------------|
| AdaBoost | 다수결을 통한 정답분류 및 오답에 가중치 부여 | - |
| GBM | 손실함수(검증지표)의 Gradient로 오답에 가중치 부여 | - |
| XGBoost | GMB대비 성능향상<br/>시스템(CPU, Mem.) 자원 효율적 사용 | 2014년 공개 |
| LightGBM | XGBoost대비 성능향상 및 자원소모 최소화<br/>XGBoost가 처리하지 못하는 대용량 데이터 학습가능<br/>근사치분할(Approximates the Split)을 통한 성능향상 | 2016년 공개 |


## Bagging vs Boosting

**1) Bagging(Bootstrap Aggregating):**   
- 부트스트래핑(Bootstraping): 예측값과 실제값의 차이 중복을 허용한 리샘플링(Resampling)  
- 페이스팅(Pasting): 이와 반대로 중복을 허용하지 않는 샘플링  


**2) Boosting:**   
- 성능이 약한 학습기(weak learner)를 여러 개 연결하여 강한 학습기(strong learner)를 만드는 앙상블 학습  
- 앞에서 학습된 모델을 보완해나가면서 더나은 모델로 학습시키는 것  

<center><img src='Image/Bagging_Boosting.png' width='700'></center> 


**3) 비교 정리:**

<center><img src='Image/Bagging_Boosting2.png' width='600'></center> 

| - | Bagging | Boosting |
|-------------|---------------------------------------|-----------------------------------------|
| 특징 | 병렬 앙상블 모델(각 모델은 서로 독립) | 연속 앙상블 모델(이전 모델의 오류 반영) |
| 목적 | Variance 감소 | Bias 감소 |
| 적합한 상황 | Low Bias + High Variance | High Bias + Low Variance |
| Sampling | Random Sampling | Random Sampling with weight on error |
